# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [ ]:
!python -m pip install semantic-kernel==0.9.6b1

In [5]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.OpenAI

In [6]:
from semantic_kernel import Kernel
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
    from semantic_kernel.utils.settings import openai_settings_from_dot_env

    api_key, org_id = openai_settings_from_dot_env()
    service_id = "oai_chat_gpt"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
    from semantic_kernel.utils.settings import azure_openai_settings_from_dot_env

    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    service_id = "aoai_chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

Let's define a prompt outlining a dialogue chat bot.


In [7]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [8]:
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [9]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [10]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [11]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Hi there! I'd be happy to help you find some book suggestions. What kind of books are you interested in? Fiction, non-fiction, mystery, romance, science fiction, or something else? Let me know your preferences!


Update the history with the output


In [12]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [13]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    chat_history.add_user_message(input_text)

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_assistant_message(str(answer))

In [14]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Certainly! If you're interested in learning about ancient Greek philosophy, you might enjoy reading "The Greeks" by H.D.F. Kitto. It provides an overview of ancient Greek culture, history, and philosophy, focusing on key figures such as Socrates, Plato, and Aristotle. Another great option is "The Birth of Western Philosophy: Socrates, Plato, and Aristotle" by Maria Kaimaki. This book delves into the lives and ideas of these influential philosophers, providing a comprehensive understanding of their contributions to philosophy.

If you're more interested in the history of Greece, "The Peloponnesian War" by Donald Kagan is a highly regarded work that explores the conflict between Athens and Sparta in the 5th century BC, shedding light on the political, military, and cultural aspects of ancient Greece.

I hope you find these suggestions helpful! Let me know if you have any other pref

In [15]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Greeks" by H.D.F. Kitto provides a comprehensive overview of ancient Greek culture, history, and philosophy. It explores the intellectual and cultural achievements of the ancient Greeks, focusing on key figures such as Socrates, Plato, and Aristotle. The book delves into various aspects of Greek life, including literature, art, politics, and religion, offering valuable insights into the development of Western civilization. It's a great resource for anyone interested in gaining a deeper understanding of ancient Greek philosophy and its impact on the world.

"The Birth of Western Philosophy: Socrates, Plato, and Aristotle" by Maria Kaimaki delves into the lives and ideas of these influential philosophers, providing a comprehensive understanding of their contributions to philosophy. The book explores their teachings, theories, and philosophical methods, shedding light on the profound influence they had on Western thought.

"Th

In [16]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read "The Greeks" by H.D.F. Kitto, you will gain a comprehensive understanding of ancient Greek history, culture, and philosophy. The book delves into the intellectual and cultural achievements of the ancient Greeks, providing insights into various aspects of Greek life, including literature, art, politics, and religion. You will learn about the key figures and events that shaped ancient Greek civilization, as well as the impact of Greek philosophy on Western thought. Additionally, the book offers valuable perspectives on the development of democracy, the city-states, and the Peloponnesian War. It's a rich exploration of the foundations of Western civilization and the enduring legacy of ancient Greece.

If you choose to read "The Birth of Western Philosophy: Socrates, Plato, and Aristotle" by Maria Kaimaki, you will gain a deep understanding of the lives and ideas of these influential philosophers

In [17]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are some more books that you might find interesting to learn about ancient Greek history and philosophy:

1. "The Histories" by Herodotus - This classic work provides a detailed account of ancient Greek history, society, and culture, as well as the Greco-Persian Wars.

2. "The Classical World: An Epic History from Homer to Hadrian" by Robin Lane Fox - This book offers a comprehensive overview of ancient Greek history, covering the period from the Bronze Age to the end of the classical era.

3. "The Hemlock Cup: Socrates, Athens and the Search for the Good Life" by Bettany Hughes - This book explores the life and influence of Socrates, offering a vivid portrayal of ancient Athens and its intellectual climate.

4. "The Intellectual Adventure of Ancient Man: An Essay on Speculative Thought in the Ancient Near East" by Henri Frankfort - While not strictly focused on Greece, this book provides insig

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [18]:
print(chat_history)

<chat_history><message role="system">You are a helpful chatbot who is good about giving book recommendations.</message><message role="assistant">Hi there! I'd be happy to help you find some book suggestions. What kind of books are you interested in? Fiction, non-fiction, mystery, romance, science fiction, or something else? Let me know your preferences!</message><message role="user">I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</message><message role="assistant">Certainly! If you're interested in learning about ancient Greek philosophy, you might enjoy reading "The Greeks" by H.D.F. Kitto. It provides an overview of ancient Greek culture, history, and philosophy, focusing on key figures such as Socrates, Plato, and Aristotle. Another great option is "The Birth of Western Philosophy: Socrates, Plato, and Aristotle" by Maria Kaimaki. This book delves into the lives and ideas of these influential philosophers, providing a comprehensive unders